# Importing Necessary Packages

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

#  Data Loading and Preprocessing


In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
num_validation_samples =0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image,label):
    image = tf.cast(image, tf.float32)
    image/= 255.
    
    return image,label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data= mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data= shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 50

train_data =  train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Model Definition


In [3]:
input_size = 784
output_size = 10

hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    
    tf.keras.layers.Dense(output_size, activation='softmax')
])

# Model Compilation and Training

In [4]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [5]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
1080/1080 - 6s - loss: 0.2320 - accuracy: 0.9328 - val_loss: 0.1197 - val_accuracy: 0.9658 - 6s/epoch - 5ms/step
Epoch 2/5
1080/1080 - 4s - loss: 0.0947 - accuracy: 0.9706 - val_loss: 0.0780 - val_accuracy: 0.9762 - 4s/epoch - 3ms/step
Epoch 3/5
1080/1080 - 4s - loss: 0.0641 - accuracy: 0.9796 - val_loss: 0.0594 - val_accuracy: 0.9832 - 4s/epoch - 3ms/step
Epoch 4/5
1080/1080 - 3s - loss: 0.0474 - accuracy: 0.9850 - val_loss: 0.0458 - val_accuracy: 0.9847 - 3s/epoch - 3ms/step
Epoch 5/5
1080/1080 - 3s - loss: 0.0384 - accuracy: 0.9877 - val_loss: 0.0399 - val_accuracy: 0.9862 - 3s/epoch - 3ms/step


# Model Evaluation


In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 766ms/step - loss: 0.0735 - accuracy: 0.9800


# Results Display

In [7]:
print('test loss: {0:.2f}%. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

test loss: 0.07%. Test accuracy: 98.00%
